In [3]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.utils.class_weight import compute_class_weight

import timm

# ================= CONFIG =================
class CFG:
    MODEL_NAME = 'swinv2_tiny_window8_256'
    IMG_SIZE = 256
    BATCH_SIZE = 8
    BASE_PATH = "/kaggle/input/aptos2019"
    TRAIN_CSV = os.path.join(BASE_PATH, "train_1.csv")
    VAL_CSV   = os.path.join(BASE_PATH, "valid.csv")
    TRAIN_DIR = os.path.join(BASE_PATH, "train_images", "train_images")
    VAL_DIR   = os.path.join(BASE_PATH, "val_images", "val_images")
    SEG_TRAIN_DIR = "/kaggle/input/aptos-2019-dataset-vessel-segmentation/segmented_outputs_train_1/segmented_outputs_train_1"
    SEG_VAL_DIR   = "/kaggle/input/aptos-2019-dataset-vessel-segmentation/segmented_outputs_val/segmented_outputs_val"
    
    S1_EPOCHS = 15; S1_LR = 1e-4; S1_USE_MIXUP = True
    S2_EPOCHS = 15; S2_LR = 3e-5; S2_USE_MIXUP = False
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    NUM_WORKERS = 2
    PATIENCE = 5
    SEED = 42
    LABEL_SMOOTHING = 0.05
    
    SAVE_PATH_S1 = "best_model_swin_stage1.pth"
    SAVE_PATH_FINAL = "best_model_swin_final.pth"

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG.SEED)

# ================= Dataset =================
class DatasetDualStream(Dataset):
    def __init__(self, df, img_dir, seg_dir, transform=None):
        self.df, self.img_dir, self.seg_dir, self.transform = df.reset_index(drop=True), img_dir, seg_dir, transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'] + '.png')
        seg_path = os.path.join(self.seg_dir, row['id_code'] + '.png')
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
        
        img = cv2.resize(img, (CFG.IMG_SIZE, CFG.IMG_SIZE))
        mask = cv2.resize(mask, (CFG.IMG_SIZE, CFG.IMG_SIZE))
        
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img, mask = augmented['image'], augmented['mask']
        
        img = A.Normalize()(image=img)['image']
        mask = (mask / 255.0)[None, :, :]
        img = ToTensorV2()(image=img)['image']
        mask = torch.tensor(mask, dtype=torch.float)
        label = torch.tensor(row['diagnosis'], dtype=torch.long)
        return img, mask, label

def get_transforms(is_train=True):
    if is_train:
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.7),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7)
        ])
    return None

# ================= Model =================
class DualStreamSwin(nn.Module):
    def __init__(self, model_name='swinv2_tiny_window8_256', num_classes=5, pretrained=True):
        super().__init__()
        self.rgb_backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, global_pool='avg')
        self.mask_backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, global_pool='avg')
        
        # 1-channel patch embed
        orig_proj = self.mask_backbone.patch_embed.proj
        new_proj = nn.Conv2d(1, orig_proj.out_channels,
                             kernel_size=orig_proj.kernel_size,
                             stride=orig_proj.stride,
                             padding=orig_proj.padding,
                             bias=(orig_proj.bias is not None))
        with torch.no_grad():
            new_proj.weight[:, 0] = orig_proj.weight.mean(dim=1)
        self.mask_backbone.patch_embed.proj = new_proj
        
        feat_dim = self.rgb_backbone.num_features + self.mask_backbone.num_features
        self.classifier = nn.Sequential(
            nn.Linear(feat_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes - 1)
        )
        
    def forward(self, rgb, mask):
        rgb_feat = self.rgb_backbone(rgb)
        mask_feat = self.mask_backbone(mask)
        fused_feat = torch.cat([rgb_feat, mask_feat], dim=1)
        return self.classifier(fused_feat)

# ================= Loss =================
class WeightedOrdinalFocalLoss(nn.Module):
    def __init__(self, num_classes=5, gamma=2.0, class_weights=None, label_smoothing=0.0):
        super().__init__()
        self.num_classes, self.gamma, self.class_weights, self.label_smoothing = num_classes, gamma, class_weights, label_smoothing
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
    def forward(self, outputs, targets):
        ordinal_targets = torch.zeros_like(outputs)
        for i, t in enumerate(targets):
            if t > 0: ordinal_targets[i, :t] = 1.0
        if self.label_smoothing > 0.0: ordinal_targets = ordinal_targets * (1.0 - self.label_smoothing) + 0.5 * self.label_smoothing
        bce = self.bce(outputs, ordinal_targets)
        if self.class_weights is not None:
            weights = self.class_weights[targets].view(-1, 1).expand(-1, outputs.shape[1])
            bce = bce * weights
        pt = torch.exp(-bce)
        focal = (1 - pt) ** self.gamma * bce
        return focal.mean()

class SmoothKappaLoss(nn.Module):
    def __init__(self, num_classes=5, eps=1e-7):
        super().__init__()
        self.num_classes, self.eps = num_classes, eps
        W = torch.zeros(num_classes, num_classes)
        for i in range(num_classes):
            for j in range(num_classes):
                W[i,j] = ((i-j)**2) / ((num_classes-1)**2)
        self.register_buffer("W", W)
    def forward(self, outputs, targets):
        device = outputs.device; B = outputs.size(0); probs = torch.sigmoid(outputs)
        class_probs = torch.zeros(B, self.num_classes, device=device)
        class_probs[:,0] = 1-probs[:,0]
        for k in range(1,self.num_classes-1):
            class_probs[:,k] = probs[:,k-1]-probs[:,k]
        class_probs[:,-1] = probs[:,-1]
        class_probs = torch.clamp(class_probs, min=1e-7, max=1.0)
        one_hot = F.one_hot(targets, num_classes=self.num_classes).float().to(device)
        conf_mat = torch.matmul(one_hot.T, class_probs)
        hist_true = one_hot.sum(dim=0); hist_pred = class_probs.sum(dim=0)
        expected = torch.outer(hist_true, hist_pred)
        W = self.W.to(device); obs = torch.sum(W*conf_mat); exp = torch.sum(W*expected)
        kappa = 1.0 - (B*obs)/(exp+self.eps)
        return 1.0 - kappa

# ================= Training Utils =================
def ordinal_to_class(outputs): return torch.sum(torch.sigmoid(outputs) > 0.5, dim=1).long()
def calculate_metrics(outputs, targets):
    preds = ordinal_to_class(outputs).cpu().numpy()
    targets_np = targets.cpu().numpy()
    return accuracy_score(targets_np, preds), cohen_kappa_score(targets_np, preds, weights='quadratic')
def clear_memory(): gc.collect(); torch.cuda.empty_cache()
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha) if alpha>0 else 1
    index = torch.randperm(x.size(0)).to(x.device)
    return lam*x + (1-lam)*x[index], y, y[index], lam

def train_epoch(model, loader, optimizer, criterion, scaler, device, use_mixup):
    model.train(); running_loss=0.0; all_out, all_t=[],[]
    for img, mask, targets in tqdm(loader, desc="Train", leave=False):
        img, mask, targets = img.to(device), mask.to(device), targets.to(device)
        optimizer.zero_grad(set_to_none=True)
        if use_mixup:
            img, targets_a, targets_b, lam = mixup_data(img, targets)
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(img, mask)
            loss = lam*criterion(outputs, targets_a)+(1-lam)*criterion(outputs, targets_b) if use_mixup else criterion(outputs, targets)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        running_loss += loss.item(); all_out.append(outputs.detach()); all_t.append(targets.detach())
    all_out, all_t = torch.cat(all_out), torch.cat(all_t)
    acc, qwk = calculate_metrics(all_out, all_t)
    return running_loss/len(loader), acc, qwk

def valid_epoch(model, loader, criterion, device):
    model.eval(); running_loss=0.0; all_out, all_t=[],[]
    with torch.no_grad():
        for img, mask, targets in tqdm(loader, desc="Valid", leave=False):
            img, mask, targets = img.to(device), mask.to(device), targets.to(device)
            outputs = model(img, mask); loss = criterion(outputs, targets)
            running_loss += loss.item(); all_out.append(outputs); all_t.append(targets)
    all_out, all_t = torch.cat(all_out), torch.cat(all_t)
    acc, qwk = calculate_metrics(all_out, all_t)
    return running_loss/len(loader), acc, qwk



def main():
    print(f"Device: {CFG.DEVICE}, Model: {CFG.MODEL_NAME} (4-Channel), Image Size: {CFG.IMG_SIZE}")
    train_df = pd.read_csv(CFG.TRAIN_CSV)
    val_df = pd.read_csv(CFG.VAL_CSV)
    
    train_tf = get_transforms(is_train=True)
    val_tf = get_transforms(is_train=False)

    train_ds = DatasetDualStream(train_df, CFG.TRAIN_DIR, CFG.SEG_TRAIN_DIR, transform=train_tf)
    val_ds   = DatasetDualStream(val_df, CFG.VAL_DIR, CFG.SEG_VAL_DIR, transform=val_tf)

    class_weights_sampler = compute_class_weight('balanced', classes=np.unique(train_df['diagnosis']), y=train_df['diagnosis'])
    sample_weights = np.array([class_weights_sampler[int(l)] for l in train_df['diagnosis']])
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, sampler=sampler, num_workers=CFG.NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CFG.BATCH_SIZE*2, shuffle=False, num_workers=CFG.NUM_WORKERS, pin_memory=True)
    
    model = DualStreamSwin(model_name='swinv2_tiny_window8_256').to(CFG.DEVICE)
    class_weights_loss = torch.tensor(class_weights_sampler, dtype=torch.float).to(CFG.DEVICE)
    focal_loss = WeightedOrdinalFocalLoss(num_classes=5, gamma=2.0, class_weights=class_weights_loss, label_smoothing=CFG.LABEL_SMOOTHING)
    kappa_loss = SmoothKappaLoss(num_classes=5)
    
    def hybrid_loss(outputs, targets): 
        return 0.7 * kappa_loss(outputs, targets) + 0.3 * focal_loss(outputs, targets)
    
    scaler = torch.cuda.amp.GradScaler()

    # --- STAGE 1 ---
    print("\n" + "="*50 + "\n     STARTING STAGE 1 (4-Channel)\n" + "="*50)
    opt = optim.AdamW(model.parameters(), lr=CFG.S1_LR, weight_decay=1e-4)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=CFG.S1_EPOCHS)
    best_val_qwk, patience_counter = -1, 0

    for epoch in range(CFG.S1_EPOCHS):
        clear_memory()
        print(f"\nEpoch {epoch+1}/{CFG.S1_EPOCHS}")
        train_loss, train_acc, train_qwk = train_epoch(model, train_loader, opt, focal_loss, scaler, CFG.DEVICE, CFG.S1_USE_MIXUP)
        val_loss, val_acc, val_qwk = validate_epoch(model, val_loader, focal_loss, CFG.DEVICE)
        sched.step()
        print(f"Train -> Loss:{train_loss:.4f} Acc:{train_acc:.4f} QWK:{train_qwk:.4f}")
        print(f"Valid -> Loss:{val_loss:.4f} Acc:{val_acc:.4f} QWK:{val_qwk:.4f}")
        if val_qwk > best_val_qwk:
            print(f"Val QWK improved from {best_val_qwk:.4f} to {val_qwk:.4f}. Saving model...")
            best_val_qwk, patience_counter = val_qwk, 0
            torch.save(model.state_dict(), CFG.SAVE_PATH_S1)
        else:
            patience_counter += 1
            if patience_counter >= CFG.PATIENCE: 
                print("Early stopping in Stage 1.")
                break
    
    # --- STAGE 2 ---
    print("\n" + "="*50 + "\n     STARTING STAGE 2 (4-Channel)\n" + "="*50)
    if os.path.exists(CFG.SAVE_PATH_S1):
        model.load_state_dict(torch.load(CFG.SAVE_PATH_S1))
    else:
        print("No Stage 1 model was saved. Continuing with the current model.")

    opt = optim.AdamW(model.parameters(), lr=CFG.S2_LR, weight_decay=1e-5)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=CFG.S2_EPOCHS)
    best_val_qwk_stage2, patience_counter = best_val_qwk, 0

    for epoch in range(CFG.S2_EPOCHS):
        clear_memory()
        print(f"\nEpoch {epoch+1}/{CFG.S2_EPOCHS}")
        train_loss, train_acc, train_qwk = train_epoch(model, train_loader, opt, hybrid_loss, scaler, CFG.DEVICE, CFG.S2_USE_MIXUP)
        val_loss, val_acc, val_qwk = validate_epoch(model, val_loader, hybrid_loss, CFG.DEVICE)
        sched.step()
        print(f"Train -> Loss:{train_loss:.4f} Acc:{train_acc:.4f} QWK:{train_qwk:.4f}")
        print(f"Valid -> Loss:{val_loss:.4f} Acc:{val_acc:.4f} QWK:{val_qwk:.4f}")
        if val_qwk > best_val_qwk_stage2:
            print(f"Val QWK improved from {best_val_qwk_stage2:.4f} to {val_qwk:.4f}. Saving final model...")
            best_val_qwk_stage2, patience_counter = val_qwk, 0
            torch.save(model.state_dict(), CFG.SAVE_PATH_FINAL)
        else:
            patience_counter += 1
            if patience_counter >= CFG.PATIENCE: 
                print("Early stopping in Stage 2.")
                break

    print(f"\nTraining Finished!\nFinal Best QWK: {best_val_qwk_stage2:.4f}")

if __name__ == "__main__":
    main()



Device: cuda, Model: swinv2_tiny_window8_256 (4-Channel), Image Size: 256


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_36/2405025121.py:239: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



     STARTING STAGE 1 (4-Channel)

Epoch 1/15


Train:   0%|          | 0/367 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

...

Train -> Loss:0.0680 Acc:0.9038 QWK:0.9726
Valid -> Loss:0.1627 Acc:0.8060 QWK:0.9034

Epoch 15/15


Train:   0%|          | 0/367 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

/tmp/ipykernel_36/1163770134.py:282: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Train -> Loss:0.0695 Acc:0.9000 QWK:0.9680
Valid -> Loss:0.1634 Acc:0.8060 QWK:0.9031

Training Finished!
Final Best QWK: 0.9057


In [6]:
import os, cv2, torch
import numpy as np, pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, cohen_kappa_score

# نفس الداتا ستراكشر بس من غير label
class DatasetDualStreamTest(Dataset):
    def __init__(self, df, img_dir, seg_dir, transform=None):
        self.df, self.img_dir, self.seg_dir, self.transform = df.reset_index(drop=True), img_dir, seg_dir, transform

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['id_code'] + '.png')
        seg_path = os.path.join(self.seg_dir, row['id_code'] + '.png')

        img = cv2.imread(img_path); img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (CFG.IMG_SIZE, CFG.IMG_SIZE))
        mask = cv2.resize(mask, (CFG.IMG_SIZE, CFG.IMG_SIZE))

        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img, mask = augmented['image'], augmented['mask']

        img = A.Normalize()(image=img)['image']
        img = ToTensorV2()(image=img)['image']
        mask = torch.tensor((mask / 255.0)[None, :, :], dtype=torch.float)

        return img, mask, row.get('diagnosis', -1), row['id_code']

# --- Load Test Data
test_df = pd.read_csv("/kaggle/input/aptos2019/test.csv")
has_labels = 'diagnosis' in test_df.columns

test_ds = DatasetDualStreamTest(
    test_df,
    "/kaggle/input/aptos2019/test_images/test_images",
    "/kaggle/input/aptos-2019-dataset-vessel-segmentation/segmented_outputs_test/segmented_outputs_test",
    transform=None
)
test_loader = DataLoader(test_ds, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=CFG.NUM_WORKERS)

# --- Define Model again (DualStreamSwin)
model = DualStreamSwin(
    model_name="swinv2_tiny_window8_256",
    num_classes=5,
    pretrained=False
).to(CFG.DEVICE)

# --- Load weights
model.load_state_dict(torch.load(CFG.SAVE_PATH_FINAL, map_location=CFG.DEVICE))
model.eval()

# --- Inference
all_preds, all_labels, all_ids = [], [], []
with torch.no_grad():
    for img_rgb, mask, labels, ids in tqdm(test_loader, desc="Test Inference"):
        img_rgb, mask = img_rgb.to(CFG.DEVICE), mask.to(CFG.DEVICE)
        outputs = model(img_rgb, mask)
        preds = ordinal_to_class(outputs).cpu().numpy()
        all_preds.extend(preds); all_ids.extend(ids)

        if has_labels:
            all_labels.extend(labels.numpy())

# --- Metrics
if has_labels:
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    qwk = cohen_kappa_score(all_labels, all_preds, weights='quadratic')

    print(f"Accuracy : {acc:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"QWK      : {qwk:.4f}")
else:
    sub = pd.DataFrame({"id_code": all_ids, "diagnosis": all_preds})
    sub.to_csv("submission.csv", index=False)
    print("Submission file saved: submission.csv")


Test Inference: 100%|██████████| 46/46 [00:25<00:00,  1.79it/s]

Accuracy : 0.8005
F1 Score : 0.6526
Recall   : 0.6710
Precision: 0.6680
QWK      : 0.9088
